In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

170508288/170498071 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 10),
 (10000, 32, 32, 3),
 (10000, 10),
 (10000, 32, 32, 3),
 (10000, 10))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(10))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('leaky-relu')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 75s 78ms/step - loss: 2.0089 - accuracy: 0.2774 - val_loss: 1.7631 - val_accuracy: 0.4435
Epoch 2/50
313/313 [==============================] - 23s 73ms/step - loss: 1.5626 - accuracy: 0.4550 - val_loss: 1.4860 - val_accuracy: 0.4796
Epoch 3/50
313/313 [==============================] - 22s 69ms/step - loss: 1.4191 - accuracy: 0.5078 - val_loss: 1.4238 - val_accuracy: 0.4931
Epoch 4/50
313/313 [==============================] - 23s 72ms/step - loss: 1.3181 - accuracy: 0.5455 - val_loss: 1.6654 - val_accuracy: 0.4259
Epoch 5/50
313/313 [==============================] - 22s 71ms/step - loss: 1.2443 - accuracy: 0.5735 - val_loss: 1.2940 - val_accuracy: 0.5475
Epoch 6/50
313/313 [==============================] - 22s 69ms/step - loss: 1.1789 - accuracy: 0.6015 - val_loss: 1.4432 - val_accuracy: 0.4979
Epoch 7/50
313/313 [==============================] - 23s 73ms/step - loss: 1.1176 - accuracy: 0.6208 - val_loss: 1.2684 - val_accuracy:

In [8]:
print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

{'loss': [1.8053257465362549, 1.5177497863769531, 1.3935015201568604, 1.3065769672393799, 1.227891206741333, 1.169546127319336, 1.1156667470932007, 1.0635063648223877, 1.0232447385787964, 0.9805762767791748, 0.9465968608856201, 0.9113465547561646, 0.8731253743171692, 0.8359682559967041, 0.8098864555358887, 0.7782467603683472, 0.7468400597572327, 0.7265978455543518, 0.6836628317832947, 0.6596941947937012, 0.6381928324699402, 0.6094450354576111, 0.5865973830223083, 0.5583140254020691, 0.5372877717018127, 0.5115222930908203, 0.4845893979072571, 0.46488896012306213, 0.4413396418094635, 0.42327144742012024, 0.3975083827972412, 0.37976861000061035, 0.36343854665756226, 0.3367063105106354, 0.3250217139720917, 0.301898717880249, 0.28773295879364014, 0.2731980085372925, 0.2638240456581116, 0.24611687660217285, 0.2406911551952362, 0.2259945273399353, 0.20914624631404877, 0.200754776597023, 0.18274718523025513, 0.17727836966514587, 0.168720543384552, 0.15578170120716095, 0.14745962619781494, 0.13